# Automatic Source Retrieval

Done:
- Ontario
- Manitoba

WIP:
- British Columbia

TODO:
- Nova Scotia
- Northwest Territories
- Nunavut
- Yukon
- Alberta
- Saskatchewan
- Prince Edward Island
- Quebec
- New Brunswick
- Newfoundland and Labrador

In [1]:
!pip install bs4
!pip install feedparser

    100% |################################| 122kB 917kB/s ta 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 194kB 2.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for feedparser ... done
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import requests
import urllib.request
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import feedparser

In [3]:
country = 'Canada'
src_cat = 'Government Website'
columns = ['start_date', 'country', 'region', 'subregion', 'source_url', 'source_category', 'source_title', 'source_full_text']

# Ontario

Since Ontario shows the most recent news on the first page, the range
will need to continue to be expanded to capture all posts. Generally ~4 pages capture ~2 weeks.

In [63]:
targets = ['https://news.ontario.ca/archive/en?page=' + str(i) for i in range(1,4)]

region = 'Ontario'
subregion = ''

# Specific structure for news.contario.ca/archive
links = []
for target in targets:
    if target.startswith('https://news.ontario.ca/archive/'):
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.findAll('article')
        
        for article in articles:
            smallersoup = BeautifulSoup(str(article), "html.parser")
            link = smallersoup.findAll('a')[0]['href']
            title = smallersoup.findAll('a')[0].string
            pub_date = smallersoup.time['datetime']
            
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            full_text = linksoup.article.text
            
            row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text


In [64]:
df = pd.DataFrame(links, columns=columns)

In [65]:
df.shape

(45, 8)

In [66]:
df.to_csv('sources/ontario.csv')

# Manitoba

Retrieve all news releases in 2020 for the Province of Manitoba.

In [67]:
url_base = 'https://news.gov.mb.ca'
targets = [url_base + '/news/index.html?month=' + str(i) + '&year=2020&day=01&bgnG=GO&d=' for i in range(1,12)]


In [70]:
%%time

region = 'Manitoba'
subregion = ''

links = []
for target in targets:
    print(target)
    if target.startswith(url_base): #manitoba
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.findAll("div", {"class": "maincontent"})
        smallersoup = BeautifulSoup(str(items), "html.parser")
        for article in smallersoup.findAll('h2'):
            a = article.a
            relative_link = a['href']
            link = url_base + relative_link.split('..')[-1]
            title = a.string
        
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            
            date_text = linksoup.findAll("span", {"class": "article_date"})[0].string
            date = pd.to_datetime(date_text, format='%B %d, %Y')
            pub_date = date.strftime('%m/%d/%Y')
            
            full_text = linksoup.findAll("div", {"class": ""})[0].text

            
            row = [pub_date, country, region, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text

https://news.gov.mb.ca/news/index.html?month=1&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=2&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=3&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=4&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=5&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=6&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=7&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=8&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=9&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=10&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=11&year=2020&day=01&bgnG=GO&d=
CPU times: user 13.3 s, sys: 374 ms, total: 13.7 s
Wall time: 3min 44s


In [47]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/manitoba.csv')
df.shape

(260, 4)

## British Columbia

In [47]:
#    'https://news.gov.bc.ca/Search?fromDate=2020%2F01%2F01&toDate=2020%2F05%2F26&q='
region = 'British Columbia'
subregion = ''

query_url = 'https://news.gov.bc.ca/Search?FromDate=01/01/2020&Page='

In [ ]:
def get_article

In [ ]:
links = []

In [50]:

page = 18
while True:
    print("Page ", page)
    target = query_url + str(page)
    response = requests.get(target)
    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.findAll("div", {"class": "article"})
    
    if not items:
        # No articles found
        break
    
    for article in items:
        smallersoup = BeautifulSoup(str(article), "html.parser")

        #for article in smallersoup.findAll('div'):

        title = smallersoup.a.string

        date_text = smallersoup.findAll("div", {"class" : "item-date"})[0].string
        date = pd.to_datetime(date_text)
        pub_date = date.strftime('%m/%d/%Y')

        link = smallersoup.a['href']

        response = requests.get(link)
        linksoup = BeautifulSoup(response.text, "html.parser")
        get_article = linksoup.findAll("article")
        if get_article:
            full_text = get_article[0].text
        else:
            print("Couldn't retrieve full text for link: ", link)
            full_text = ""

        row = [pub_date, country, region, link, src_cat, title, full_text]
        links.append(row)

    page += 1

Page  18
Page  19
Page  20
Page  21
Page  22
Page  23
Page  24
Page  25
Page  26
Page  27
Page  28
Page  29
Page  30
Page  31
Page  32
Page  33
Page  34
Page  35
Page  36
Page  37
Page  38
Page  39
Page  40
Page  41
Page  42
Page  43
Page  44
Page  45
Page  46
Page  47
Page  48
Page  49
Page  50
Page  51
Page  52
Page  53
Page  54
Page  55
Page  56
Page  57
Page  58
Page  59
Page  60
Page  61
Page  62
Page  63
Page  64
Page  65
Page  66
Page  67
Page  68
Page  69
Page  70
Page  71
Page  72
Page  73


In [52]:
links

[['05/29/2020',
  'Canada',
  'British Columbia',
  'https://news.gov.bc.ca/releases/2020HLTH0170-000978',
  'Government Website',
  "Joint statement on B.C.'s COVID-19 response, latest updates",
  '\n\nAdrian\xa0Dix, Minister of Health, and Dr. Bonnie Henry, B.C.’s provincial health officer, have issued the following joint statement regarding updates on the novel coronavirus (COVID-19) response in British Columbia:“Today, we are announcing four new cases, for a total of 2,562 cases in British Columbia.“There are 228 active cases of COVID-19 in the province and 2,170 people who tested positive have recovered.“Of the total COVID-19 cases, 34 individuals are hospitalized, six of whom are in intensive care. The remaining people with COVID-19 are recovering at home in self-isolation.“Every health region in British Columbia has patients with COVID-19. Since the start of the pandemic, there have been 900 in the Vancouver Coastal Health region, 1,277 in the Fraser Health region, 127 in the Is

In [51]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/britishcolumbia.csv')
df.shape

AssertionError: 8 columns passed, passed data had 7 columns

In [45]:
items

[]

In [37]:
article = items[0]
smallersoup = BeautifulSoup(str(article), "html.parser")
smallersoup

<div class="article">
<h2><a href="https://news.gov.bc.ca/releases/2020HLTH0170-000978">Joint statement on B.C.'s COVID-19 response, latest updates</a></h2>
<div class="item-date">Friday, May 29, 2020 5:50 PM</div>
<div class="item-content">
<div class="asset flickr"><a href="https://news.gov.bc.ca/releases/2020HLTH0170-000978"><img src="https://media.news.gov.bc.ca/proxy?url=https%3A%2F%2Flive.staticflickr.com%2F65535%2F49927945267_040ba7dde1_b.jpg&amp;token=baFsabrlcxTMo2KBNjbBGtnrML%2BXGxRfk0F31oV1yD4%3D"/></a></div>
<div class="item-text with-image">
<p>Adrian Dix, Minister of Health, and Dr. Bonnie Henry, B.C.’s provincial health officer, have issued the following joint statement regarding updates on the novel coronavirus (COVID-19) response in British Columbia:</p>
<a class="read-more" href="https://news.gov.bc.ca/releases/2020HLTH0170-000978">Read More</a>
</div>
</div>
<div class="clear"></div>
</div>

In [34]:
link = smallersoup.a['href']
link

'https://news.gov.bc.ca/releases/2020HLTH0170-000978'

In [36]:
title = smallersoup.a.string
title

"Joint statement on B.C.'s COVID-19 response, latest updates"

In [33]:
date_text = smallersoup.findAll("div", {"class" : "item-date"})[0].string
date_text

'Friday, May 29, 2020 5:50 PM'

In [42]:
links = []

#for article in smallersoup.findAll('div'):

title = smallersoup.a.string

date_text = smallersoup.findAll("div", {"class" : "item-date"})[0].string
date = pd.to_datetime(date_text)
pub_date = date.strftime('%m/%d/%Y')

link = smallersoup.a['href']

response = requests.get(link)
linksoup = BeautifulSoup(response.text, "html.parser")
full_text = linksoup.findAll("article")[0].text

row = [pub_date, country, region, link, src_cat, title, full_text]
links.append(row)

'05/29/2020'